# 加密货币指数计算演示

本演示将逐步展示如何构建和计算加密货币市值加权指数。

> **注意**：运行前请先参考 [`../README.md`](../README.md) 配置环境和导入必要模块。

## 市值加权指数算法原理

### 什么是市值加权指数？

市值加权指数（Market Capitalization Weighted Index）是一种常见的指数编制方法，广泛应用于股票市场和加密货币市场。在这种方法中，每个成分资产在指数中的权重由其市值大小决定。

### 核心概念

**市值（Market Capitalization）**：指某个资产的总市场价值，计算公式为：
- 市值 = 当前价格 × 流通供应量

**权重（Weight）**：指某个资产在指数中所占的比例，计算公式为：
- 权重 = 该资产市值 / 所有成分资产市值总和

### 算法基本流程

1. **确定成分资产**：选择要纳入指数的资产（如前10大、前20大等）
2. **计算各资产市值**：获取每个资产的当前价格和流通供应量
3. **计算权重**：根据市值占比确定每个资产的权重
4. **设定基准值**：选择一个基准日期和基准指数值（通常为1000或100）
5. **计算指数值**：根据权重和价格变动计算当前指数值

### 指数计算公式

**基准日指数值**：
```
指数值 = 基准值（如1000）
```

**后续日期指数值**：
```
指数值 = 基准值 × (当前总市值 / 基准日总市值)
```

其中，总市值为所有成分资产市值之和。

### 市值加权的优势

1. **客观性**：权重完全基于市场价值，无需主观判断
2. **代表性**：大市值资产对指数影响更大，更能反映市场整体状况
3. **可投资性**：投资者可以按权重比例构建投资组合
4. **广泛认可**：标普500、纳斯达克等主要指数都采用此方法

### 在加密货币中的应用

加密货币市值加权指数通常会：
- 排除稳定币（如USDT、USDC）
- 排除包装币和衍生品（如WBTC、stETH）
- 只包含原生代币
- 定期调整成分币种以反映市场变化

## 步骤0：环境准备

在获取数据之前，需要进行如下环境设置。

提示：
- 本演示使用本地数据库 data/market.db；请先完成数据导入或更新。
- 为保持输出整洁，初始化阶段临时关闭日志，实例化完成后会恢复。

In [ ]:
# 环境设置
import sys
from pathlib import Path
import logging

# 项目路径设置
project_root = Path.cwd().parent.parent  
sys.path.insert(0, str(project_root))
print(f"✅ 项目路径: {project_root}")

# 导入模块
from src.classification.unified_classifier import UnifiedClassifier
from src.downloaders.daily_aggregator import DailyDataAggregator

# 指定使用数据库（更短的文件名）
working_db_path = project_root / 'data' / 'market.db'
print(f"🔧 使用数据库: {working_db_path.name} ({working_db_path.stat().st_size / 1024 / 1024:.1f}MB)")

# 临时禁用所有日志输出，避免初始化过程中的INFO日志干扰
logging.disable(logging.CRITICAL)

# 创建实例，启用数据库模式并指定数据库路径
aggregator = DailyDataAggregator(use_database=True, db_path=str(working_db_path))
classifier = UnifiedClassifier()

# 重新启用日志，以便后续操作可以正常输出错误和警告
logging.disable(logging.NOTSET)

print("🎉 环境设置完成！数据库模式已启用")

## 步骤1：获取基准日数据

目的：获取基准日市场数据，用于确定指数成分与初始权重。

现在我们开始获取和处理代币数据。我们会使用项目中的数据聚合器来获取指定日期的市场数据。

In [ ]:
# 配置参数
base_date = "2023-12-01"  # 基准日期 - 可以修改为任意有数据的日期
coin_num = 15  # 要获取的代币数量

# 导入显示工具
from src.utils.display_utils import CryptoDataDisplayer
from src.utils.database_utils import DatabaseManager

# 初始化数据库管理器
db = DatabaseManager(str(working_db_path))

print(f"🎯 基准日期: {base_date}")
print(f"📊 代币数量: {coin_num}")

# 获取并处理数据 - 静默处理过程
import contextlib
import io

with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
    raw_data = aggregator.get_daily_data(base_date)
    displayer = CryptoDataDisplayer()
    crypto_data = displayer.clean_data(raw_data)

if len(crypto_data) > 0:
    # 显示最终结果
    displayer.show_table(crypto_data, 
                        top_n=coin_num,
                        title=f"前{coin_num}大市值币种 (基准日期: {base_date})")
else:
    print("❌ 没有获取到任何数据")

## 步骤2：计算指数权重

目的：基于基准日成分的市值占比计算权重，权重之和应≈1。

在获得筛选后的代币数据后，我们需要计算每个代币在指数中的权重。权重基于市值占比，市值越大的代币在指数中的权重越高。

In [ ]:
import pandas as pd

# 选择前N大市值币种作为指数成分
top_coins = crypto_data.head(coin_num).copy()

# 计算权重
total_market_cap = top_coins['market_cap'].sum()
top_coins['weight'] = top_coins['market_cap'] / total_market_cap

print(f"🎯 指数成分币种: {len(top_coins)} 个")
print(f"💰 总市值: ${total_market_cap:,.0f}")

# 验证权重总和
weight_sum = top_coins['weight'].sum()
print(f"✅ 权重验证: {weight_sum:.6f} (应该接近1.0)")

# 将权重转换为百分比显示（CryptoDataDisplayer会自动格式化weight列为百分比）
weight_data = top_coins.copy()
weight_data['weight'] = weight_data['weight'] * 100  # 转换为百分比数值

# 使用 CryptoDataDisplayer 显示权重分布表格
print(f"\n📊 权重分布:")
displayer.show_table(weight_data, 
                     columns=['rank', 'symbol', 'name', 'price', 'market_cap', 'weight'],
                     title=f"前{coin_num}大币种权重分布",
                     top_n=coin_num)

## 步骤3：设定基准指数值

目的：设定基准指数值（如1000）并记录基准日总市值，用于后续比例计算。

为了计算指数，我们需要设定一个基准日期和基准指数值。通常会选择一个具有代表性的日期作为基准日，并设定基准指数值为1000（这是金融市场的常见做法）。

本演示使用步骤1设置的 base_date 作为基准日，指数基准值设为 1000。

In [ ]:
# 设定基准参数（使用步骤1中设定的基准日期）
base_index_value = 1000  # 基准指数值
base_total_market_cap = total_market_cap  # 基准日总市值

print(f"📅 基准日期: {base_date}")
print(f"📊 基准指数值: {base_index_value}")
print(f"💰 基准总市值: ${base_total_market_cap:,.0f}")

# 创建指数基本信息
index_info = {
    'name': f'Crypto{coin_num} 市值加权指数',
    'base_date': base_date,
    'base_value': base_index_value,
    'base_market_cap': base_total_market_cap,
    'constituents_count': len(top_coins),
    'constituents': top_coins[['symbol', 'name', 'weight']].to_dict('records')
}

print(f"\n🎯 指数信息:")
print(f"  指数名称: {index_info['name']}")
print(f"  成分币种: {index_info['constituents_count']} 个")
print(f"  基准日期: {index_info['base_date']}")
print(f"  基准指数值: {index_info['base_value']}")

# 当前日期的指数值（由于是基准日，所以就是基准值）
current_index_value = base_index_value
print(f"\n📈 当前指数值: {current_index_value}")
print(f"📈 指数涨跌: +0.00% (基准日)")

# 显示指数构成详情 - 使用统一的表格格式
print(f"\n💡 指数构成 (权重 > 1%的币种):")
significant_weights = top_coins[top_coins['weight'] > 0.01].copy()  # 权重大于1%

# 将权重转换为百分比显示
significant_weights['weight'] = significant_weights['weight'] * 100

# 使用 CryptoDataDisplayer 显示构成详情表格
displayer.show_table(significant_weights, 
                    columns=['rank', 'symbol', 'name', 'weight'],
                    title=f"指数构成详情 (权重 > 1%)",
                    top_n=len(significant_weights))

## 步骤4：计算目标日指数值

为了演示指数是如何随时间变化的，我们可以计算指定日期的指数值。指数计算的基本公式是：

**指数值 = 基准值 × (当前总市值 / 基准总市值)**

假设与边界：
- 成分固定：目标日按基准日选出的成分计算，不做当日按市值重选（不含动态再平衡）。
- 数据缺失：若目标日某成分缺数据，则按可获得的数据计算总市值，可能导致轻微偏差。
- 清洗约定：稳定币、包装币等过滤在上游清洗阶段完成，本演示不重复说明。

让我们计算一个具体日期的指数值。

In [62]:
# 配置要计算的日期
target_date = "2024-07-01"  # 可以修改为任意有数据的日期

print(f"🎯 计算日期: {target_date}")

try:
    # 静默获取指定日期的数据
    with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
        date_data = aggregator.get_daily_data(target_date)
        clean_date_data = displayer.clean_data(date_data)
    
    # 获取相同的成分币种数据
    base_symbols = set(top_coins['symbol'].tolist())
    current_constituents = clean_date_data[clean_date_data['symbol'].isin(base_symbols)]
    
    # 计算当日总市值
    current_total_market_cap = current_constituents['market_cap'].sum()
    
    # 计算指数值
    index_value = base_index_value * (current_total_market_cap / base_total_market_cap)
    
    # 计算相对基准日的变化
    change_points = index_value - base_index_value
    change_percent = (change_points / base_index_value) * 100
    
    print(f"\n📊 {index_info['name']} 指数计算结果:")
    print(f"  基准日期 ({base_date}): {base_index_value}")
    print(f"  计算日期 ({target_date}): {index_value:.2f}")
    print(f"  指数变化: {change_points:+.2f} 点 ({change_percent:+.2f}%)")
    
except Exception as e:
    print(f"❌ 计算失败: {target_date} 日期可能没有数据")

print(f"\n✅ 指数计算演示完成！")

🎯 计算日期: 2024-07-01

📊 Crypto15 市值加权指数 指数计算结果:
  基准日期 (2023-12-01): 1000
  计算日期 (2024-07-01): 1676.46
  指数变化: +676.46 点 (+67.65%)

✅ 指数计算演示完成！


## 总结

通过以上演示，我们完整展示了加密货币市值加权指数的计算过程：

### 关键步骤回顾

1. **数据获取与清理**：从历史数据中获取代币的价格和市值信息
2. **权重计算**：基于市值计算每个代币在指数中的权重
3. **基准设定**：确定基准日期和基准指数值
4. **指数计算**：使用公式计算不同日期的指数值

### 核心公式

```
指数值 = 基准值 × (当前总市值 / 基准总市值)
权重 = 该币种市值 / 所有成分币种总市值
```